In [1]:
import pandas as pd
import numpy as np
import csv
from datetime import datetime
from datetime import date
import calendar
import requests
import pyodbc
import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_login():
	server = ''
	db = ''
	uid = ''
	passwd = ''
	with open("azure.txt") as f:
		lines = f.read().split("\n")
		for i in lines:
			if i.split("=")[0] == "server":
				server = i.split("=")[1]
			if i.split("=")[0] == "db":
				db = i.split("=")[1]
			if i.split("=")[0] == "uid":
				uid = i.split("=")[1]
			if i.split("=")[0] == "passwd":
				passwd = i.split("=")[1]	
	return (server, db, uid, passwd)

In [3]:
def get_sheet(spreadsheetID, sheetID, create_date, save):
	url = f"https://docs.google.com/spreadsheets/d/{spreadsheetID}/gviz/tq?tqx=out:csv&gid={sheetID}"
	res = requests.get(url)
	if save == True:
		with open(f"vintage-metagame-rawdata-{create_date}.csv", "wb") as f:
			f.write(res.content)
	return pd.read_csv(url)

In [14]:
def clean_merged_data(create_date, save, raw):
	vintage = raw[raw.columns[:11]]
	vintage.columns = ['rank','player','wins','losses','byes','arch','subarch','deck','details','date','event_type']

	# Replace NA values in 'byes' column with 0.
	vintage.byes = vintage.byes.fillna(0)
	vintage.byes = vintage.byes.astype("int")

	# Propagate 'event_type' data to each record.
	vintage["event_type"].replace({"Showcase Qualifier": "Showcase_Qualifier"}, inplace=True)

	event_type = vintage.event_type.tolist()
	for index,i in enumerate(event_type):
		if isinstance(i, str):
			new = i
		else:
			event_type[index] = new
	
	vintage["event_type"] = event_type

	event_cnt = 1
	date_last = ''
	etype_last = ''
	event_id = []
	dates_new = []
	for index, row in vintage.iterrows():
		month = row['date'].split("/")[0].zfill(2)
		day = row['date'].split("/")[1].zfill(2)
		year = row['date'].split("/")[2]
		if row['rank'] == 1:
			if (f'20{year}-{month}-{day}' == date_last) and (row['event_type'] == etype_last):
				event_cnt += 1
			if (f'20{year}-{month}-{day}' != date_last) or (row['event_type'] != etype_last):
				event_cnt = 1
		etype = row['event_type']
		event_id.append(f'20{year}-{month}-{day}-{etype}-{event_cnt}')
		dates_new.append(f'20{year}-{month}-{day}')
		date_last = f'20{year}-{month}-{day}'
		etype_last = row['event_type']
	vintage["event_id"] = event_id

	# Create a second table called Events. Remove duplicate records such that each row represents a unique event.
	events = pd.DataFrame({"event_id" : event_id, "event_type" : event_type, "date" : dates_new})
	events = events.groupby(["event_id"], as_index=False)["event_type", "date"].last()

	# Add 'entries' column to Events table to represents number of players in each event.
	players = vintage.groupby(["event_id"], as_index=False)["rank"].max()

	events = events.merge(players, on="event_id")
	events.rename(columns={"rank" : "entries", "date" : "event_date"}, inplace=True)

	# Add 'day_of_week' column to Events table.
	events["day_of_week"] = events["event_date"].apply(lambda x: calendar.day_name[datetime.strptime(x, "%Y-%m-%d").weekday()])

	# Drop 'details' column. Drop 'date' and 'event_type' columns that are now in the Events table.
	vintage.drop(["details"], axis=1, inplace=True)
	vintage.drop(["date"], axis=1, inplace=True)
	vintage.drop(["event_type"], axis=1, inplace=True)

	# Rename 'rank' column to 'finish'.
	vintage.rename(columns={"rank" : "finish"}, inplace=True)

	# Replace commas and quotes because it breaks importing with SQL commands.
	vintage["arch"] = vintage["arch"].apply(lambda x: str(x).replace(",", ""))
	vintage["subarch"] = vintage["subarch"].apply(lambda x: str(x).replace(",", ""))
	vintage["deck"] = vintage["deck"].apply(lambda x: str(x).replace(",", ""))
	
	vintage['arch'] = vintage['arch'].apply(lambda x: x.replace("'", ''))
	vintage['subarch'] = vintage['subarch'].apply(lambda x: x.replace("'", ''))
	vintage['deck'] = vintage['deck'].apply(lambda x: x.replace("'", ''))
	
	# Save and export to CSV.
	if save == True:
		vintage.to_csv(f"vintage-results-{create_date}.csv", index=False)
		events.to_csv(f"vintage-events-{create_date}.csv", index=False)
	return (vintage, events)

In [17]:
create_date = date.today().strftime("%Y-%m-%d")
df = get_sheet("1wxR3iYna86qrdViwHjUPzHuw6bCNeMLb72M25hpUHYk", "1693401931", create_date, save=False)
vintage, events = clean_merged_data(create_date=create_date, save=False, raw=df)

In [18]:
server, db, uid, passwd = get_login()

# Connect to the database
conn_str = 'DRIVER={SQL Server};SERVER=tcp:' + f'{server};PORT=1433;DATABASE={db};UID={uid};PWD={passwd}'
print(conn_str)
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

events_added = 0
events_updated = 0
results_added = 0
results_updated = 0
for index, row in events.iterrows():
    values = tuple(row.tolist())
    event_id = row['event_id']
    event_type = row['event_type']
    event_date = row['event_date']
    entries = row['entries']
    day_of_week = row['day_of_week']
    try:
        cursor.execute('INSERT INTO vintage_events (event_id, event_type, event_date, entries, day_of_week) VALUES (?, ?, ?, ?, ?)', values)
        events_added += 1
    except:
        sql = f"UPDATE vintage_events SET event_type = '{event_type}', event_date = '{event_date}', entries = {entries}, day_of_week = '{day_of_week}' WHERE event_id = '{event_id}'"
        cursor.execute(sql)
        events_updated += 1

for index, row in vintage.iterrows():
    values = tuple(row.tolist())
    finish = row['finish']
    player = row['player']
    wins = row['wins']
    losses = row['losses']
    byes = row['byes']
    arch = row['arch']
    subarch = row['subarch']
    deck = row['deck']
    event_id = row['event_id']
    try:
        cursor.execute('INSERT INTO vintage_results (finish, player, wins, losses, byes, arch, subarch, deck, event_id) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)', values)
        results_added += 1
    except:
        sql = f"UPDATE vintage_results SET player = '{player}', wins = {wins}, losses = {losses}, byes = {byes}, arch = '{arch}', subarch = '{subarch}', deck = '{deck}' WHERE finish = {finish} AND event_id = '{event_id}'"
        print(sql)
        cursor.execute(sql)
        results_updated += 1

print(f'events added: {events_added}, skipped: {events_updated}')
print(f'results added: {results_added}, skipped: {results_updated}')

# Commit the transaction and close the connection
conn.commit()
conn.close()

DRIVER={SQL Server};SERVER=tcp:sundodger.database.windows.net;PORT=1433;DATABASE=sundodgerdb;UID=chriserickson;PWD=Erickson10
UPDATE vintage_results SET player = 'sixmp', wins = 8, losses = 1, byes = 0, arch = 'Combo', subarch = 'Doomsday', deck = 'nan' WHERE finish = 1 AND event_id = '2022-12-25-Challenge-1'
UPDATE vintage_results SET player = 'JOANANTON', wins = 7, losses = 2, byes = 0, arch = 'Blue Tinker', subarch = 'PO', deck = 'Esper Saga PO' WHERE finish = 2 AND event_id = '2022-12-25-Challenge-1'
UPDATE vintage_results SET player = 'Vertyx_', wins = 7, losses = 1, byes = 0, arch = 'Combo', subarch = 'Doomsday', deck = 'nan' WHERE finish = 3 AND event_id = '2022-12-25-Challenge-1'
UPDATE vintage_results SET player = 'yoshiwata', wins = 6, losses = 2, byes = 0, arch = 'DRS', subarch = '4c DRS Walkers', deck = 'nan' WHERE finish = 4 AND event_id = '2022-12-25-Challenge-1'
UPDATE vintage_results SET player = 'medvedev', wins = 5, losses = 2, byes = 0, arch = 'Combo', subarch = 'Oop

KeyboardInterrupt: 